<a href="https://colab.research.google.com/github/Satyam296/Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install faiss-cpu
!pip install annoy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553318 sha256=14f29efc29213ddfecf9d987fb3db9132265d53af6fe8200032aeaac01518668
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

In [8]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.8 MB/s eta 0:00:00


In [9]:
def load_pdf(data_path):
    loader = PyPDFLoader(data_path)
    documents = loader.load()
    return documents

extracted_data = load_pdf("/content/KnowledgeBase.pdf")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"Number of chunks: {len(text_chunks)}")

Number of chunks: 91


In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


embeddings = download_hugging_face_embeddings()

query_result = embeddings.embed_query("Hello world")
print(f"Query Embedding Length: {len(query_result)}")

<ipython-input-10-d4526605a793>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Query Embedding Length: 384


In [12]:
docsearch = FAISS.from_texts([t.page_content for t in text_chunks], embeddings)
query = "What is Navshikhsa?"
docs = docsearch.similarity_search(query, k=3)
print("Top 3 Documents for the Query:", docs)

Top 3 Documents for the Query: [Document(id='dcac960b-8e98-4a91-9f9f-85277e39faaa', metadata={}, page_content='Q: What is Nav Shiksha? \nA: Nav Shiksha is an innovative education platform designed to connect students, teachers, and \nparents while fostering personalized learning and skill development.\nQ: Who can use Nav Shiksha? \nA: Nav Shiksha is for students, parents, and teachers, offering unique features for each to improve \nlearning outcomes and collaboration.\nQ: What makes Nav Shiksha different from other educational platforms?'), Document(id='63f3cf39-777c-4345-bdd7-98051e7cb647', metadata={}, page_content='A: It analyzes your academic history, interests, and aptitudes to suggest courses and resources \ntailored just for you.\nQ: How does Nav Shiksha help students? \nA: It offers personalized courses, a doubt-solving section, interactive learning tools, and progress \ntracking to enhance student success.\nQ: Can I track my performance on Nav Shiksha? \nA: Absolutely! Nav Shi